In [59]:
import pandas as pd
import numpy as np
import pickle
import json
import ast

In [60]:
game_data = pd.read_csv('../data/steamspy_data.csv')

In [61]:
game_data.head(50)

,appid,name,developer,publisher,score_rank,positive,negative,userscore,owners,average_forever,average_2weeks,median_forever,median_2weeks,price,initialprice,discount,languages,genre,ccu,tags
0,10,Counter-Strike,Valve,Valve,NaN,185686,4807,0,"10,000,000 .. 20,000,000",9363,426,262,323,199,999,80,"English, French, German, Italian, Spanish - Sp...",Action,11955,"{'Action': 5372, 'FPS': 4796, 'Multiplayer': 3..."
1,20,Team Fortress Classic,Valve,Valve,NaN,5235,874,0,"2,000,000 .. 5,000,000",852,3,27,3,99,499,80,"English, French, German, Italian, Spanish - Sp...",Action,94,"{'Action': 745, 'FPS': 306, 'Multiplayer': 258..."
2,30,Day of Defeat,Valve,Valve,NaN,4885,541,0,"5,000,000 .. 10,000,000",811,0,16,0,99,499,80,"English, French, German, Italian, Spanish - Spain",Action,119,"{'FPS': 785, 'World War II': 246, 'Multiplayer..."
3,40,Deathmatch Classic,Valve,Valve,NaN,1791,403,0,"5,000,000 .. 10,000,000",271,0,12,0,99,499,80,"English, French, German, Italian, Spanish - Sp...",Action,10,"{'Action': 628, 'FPS': 138, 'Classic': 106, 'M..."
4,50,Half-Life: Opposing Force,Gearbox Software,Valve,NaN,12501,638,0,"5,000,000 .. 10,000,000",1919,3,171,5,99,499,80,"English, French, German, Korean",Action,122,"{'FPS': 879, 'Action': 321, 'Classic': 250, 'S..."
5,60,Ricochet,Valve,Valve,NaN,3583,823,0,"5,000,000 .. 10,000,000",228,0,3,0,99,499,80,"English, French, German, Italian, Spanish - Sp...",Action,8,"{'Action': 585, 'FPS': 128, 'Multiplayer': 103..."
6,70,Half-Life,Valve,Valve,NaN,65013,2344,0,"5,000,000 .. 10,000,000",1241,58,156,60,199,999,80,"English, French, German, Italian, Spanish - Sp...",Action,894,"{'FPS': 2214, 'Sci-fi': 1749, 'Action': 1746, ..."
7,80,Counter-Strike: Condition Zero,Valve,Valve,NaN,18108,1771,0,"5,000,000 .. 10,000,000",1420,1,32,1,199,999,80,"English, French, German, Italian, Spanish - Sp...",Action,488,"{'Action': 1356, 'FPS': 1010, 'Shooter': 745, ..."
8,130,Half-Life: Blue Shift,Gearbox Software,Valve,NaN,8929,868,0,"10,000,000 .. 20,000,000",1920,71,121,71,99,499,80,"English, French, German",Action,55,"{'FPS': 454, 'Action': 281, 'Sci-fi': 211, 'Si..."
9,220,Half-Life 2,Valve,Valve,NaN,128896,3490,0,"10,000,000 .. 20,000,000",989,387,380,387,199,999,80,"English, French, German, Italian, Korean, Span...",Action,1041,"{'FPS': 3844, 'Action': 2731, 'Sci-fi': 2378, ..."


In [62]:
game_data = game_data[['appid', 'name', 'genre', 'tags']]

In [63]:
game_data.head()

,appid,name,genre,tags
0,10,Counter-Strike,Action,"{'Action': 5372, 'FPS': 4796, 'Multiplayer': 3..."
1,20,Team Fortress Classic,Action,"{'Action': 745, 'FPS': 306, 'Multiplayer': 258..."
2,30,Day of Defeat,Action,"{'FPS': 785, 'World War II': 246, 'Multiplayer..."
3,40,Deathmatch Classic,Action,"{'Action': 628, 'FPS': 138, 'Classic': 106, 'M..."
4,50,Half-Life: Opposing Force,Action,"{'FPS': 879, 'Action': 321, 'Classic': 250, 'S..."


In [64]:
game_data.set_index('appid', inplace=True)

In [65]:
game_data

,name,genre,tags
appid,,,
10,Counter-Strike,Action,"{'Action': 5372, 'FPS': 4796, 'Multiplayer': 3..."
20,Team Fortress Classic,Action,"{'Action': 745, 'FPS': 306, 'Multiplayer': 258..."
30,Day of Defeat,Action,"{'FPS': 785, 'World War II': 246, 'Multiplayer..."
40,Deathmatch Classic,Action,"{'Action': 628, 'FPS': 138, 'Classic': 106, 'M..."
50,Half-Life: Opposing Force,Action,"{'FPS': 879, 'Action': 321, 'Classic': 250, 'S..."
...,...,...,...
1483870,Draw & Guess,"Casual, Indie","{'Casual': 264, 'Multiplayer': 245, 'Hand-draw..."
1517290,Battlefield 2042,"Action, Adventure, Casual","{'Shooter': 445, 'Action': 407, 'Multiplayer':..."
1520470,封灵档案,Free to Play,"{'Sexual Content': 502, 'Free to Play': 342, '..."


In [66]:
def get_tags(tag_dict):
    tags = ast.literal_eval(tag_dict)
    tag_list = []
    for tag in tags:
        tag_list.append(tag)
        
    if len(tag_list) > 10:
        return tag_list[:10]
    else:
        return tag_list

In [67]:
game_data['tags'] = game_data['tags'].apply(get_tags)

In [68]:
game_data.tail(50)

,name,genre,tags
appid,,,
1220010,Home Behind 2,"Adventure, Indie, RPG, Simulation, Strategy, E...","[Strategy, RPG, Adventure, Indie, Simulation, ..."
1222140,Detroit: Become Human,"Action, Adventure","[Choices Matter, Story Rich, Multiple Endings,..."
1222670,The Sims 4,"Casual, Simulation","[Life Sim, Character Customization, Simulation..."
1222680,Need for Speed Heat,"Action, Adventure, Racing, Sports","[Racing, Open World, Multiplayer, Driving, Act..."
1222700,A Way Out,"Action, Adventure, Indie","[Co-op, Online Co-Op, Story Rich, Split Screen..."
1225330,NBA 2K21,"Simulation, Sports","[Basketball, Sports, Simulation, Character Cus..."
1226470,Shadow Arena,"Action, Free to Play, Massively Multiplayer, R...","[Action, Free to Play, Battle Royale, Massivel..."
1229490,ULTRAKILL,"Action, Indie, Early Access","[Early Access, FPS, Arena Shooter, Spectacle f..."
1237950,STAR WARS Battlefront II,"Action, Adventure","[Multiplayer, FPS, Singleplayer, Shooter, Thir..."


In [69]:
len(game_data)

1000

In [70]:
library_df = pd.read_csv('../data/library_data.csv')
library_df.head(50)

,steamid,library
0,76561198219067393,"[{'appid': 220, 'name': 'Half-Life 2', 'hours'..."
1,76561198148157441,"[{'appid': 17390, 'name': 'Spore', 'hours': 26..."
2,76561198993539076,hidden
3,76561198247182340,hidden
4,76561198278705159,hidden
5,76561198306000904,hidden
6,76561199041871881,hidden
7,76561198398210058,hidden
8,76561198313209867,hidden
9,76561198170079242,"[{'appid': 3830, 'name': 'Psychonauts', 'hours..."


In [12]:
library_df['library'].value_counts()

hidden                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  

In [71]:
def get_tags(appid):
    return game_data.loc[appid]['tags']

In [72]:
get_tags(10)

['Action',
 'FPS',
 'Multiplayer',
 'Shooter',
 'Classic',
 'Team-Based',
 'First-Person',
 'Competitive',
 'Tactical',
 "1990's"]

In [73]:
def tag_hours(games_list):
    games_list = ast.literal_eval(games_list)
    tag_dict = {}
    for game_dict in games_list:
        if game_dict['appid'] in game_data.index and game_dict['hours'] != 0:            
            tags = get_tags(game_dict['appid'])
            for tag in tags:
                if tag in tag_dict.keys():
                    tag_dict[tag] += game_dict['hours']
                else:
                    tag_dict[tag] = game_dict['hours']
    return dict(sorted(tag_dict.items(), key=lambda item: item[1], reverse=True))
        

In [74]:
print(tag_hours(library_df.iloc[0]['library']))

{'Multiplayer': 425861, 'First-Person': 368655, 'PvP': 316982, 'Survival': 290547, 'Open World': 287817, 'Building': 269759, 'Sandbox': 261696, 'Adventure': 249079, 'Open World Survival Craft': 221824, 'Crafting': 221824, 'Action': 174003, 'Shooter': 165997, 'FPS': 105497, 'Tactical': 88505, 'Co-op': 76772, 'Team-Based': 62579, 'Competitive': 55175, 'Funny': 49565, 'Physics': 49438, 'Comedy': 48358, 'Moddable': 47849, 'Mod': 47849, 'Online Co-Op': 41898, 'Horror': 40691, 'e-sports': 40442, 'Battle Royale': 38147, 'Zombies': 33174, 'Simulation': 32051, 'Third-Person Shooter': 31681, 'Massively Multiplayer': 30155, 'Crime': 25875, 'Third Person': 25493, 'Automobile Sim': 25101, 'Mature': 25082, 'Hero Shooter': 24370, 'Free to Play': 11177, 'Survival Horror': 7517, 'Singleplayer': 6373, 'Realistic': 4982, 'Military': 4906, 'Looter Shooter': 4430, 'Post-apocalyptic': 4282, 'Story Rich': 4083, 'War': 4011, 'Memes': 3178, 'World War II': 3132, 'RPG': 2761, 'Atmospheric': 2375, 'Base-Building

In [ ]:
def rate_games(steamid, appid):
    
    
    

In [75]:
get_tags(620)

['Platformer',
 'Puzzle',
 'Dark Humor',
 'Puzzle-Platformer',
 'First-Person',
 'Story Rich',
 'Female Protagonist',
 'Funny',
 '3D Platformer',
 'Action-Adventure']

In [42]:
library_df['library'] = library_df['library'].apply(build_profile)
library_df

ValueError: malformed node or string: <_ast.Name object at 0x000001FF72B40100>